In [ ]:
import pandas as pd
import numpy as np
import bisect
import seaborn as sns
import scipy
import scipy.cluster.hierarchy as sch
import plotly.express as px
import matplotlib.pyplot as plt
import networkx as nx

from itertools import chain
from math import pi
from sklearn import preprocessing
from GGLasso.gglasso.problem import glasso_problem
from utils import transform_features, scale_array_by_diagonal
from utils import PCA

from scipy import stats
from scipy.spatial import distance

from networkx.utils import cuthill_mckee_ordering

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, HoverTool, LabelSet, PointDrawTool
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import RdBu, Blues8
from bokeh.models import HoverTool, Panel, Tabs, ColorBar, LinearColorMapper
from bokeh.layouts import row

In [ ]:
#count table
# raw = pd.read_csv('data/composition_feature-table.tsv', sep='\t', index_col = 0)

raw = pd.read_csv("~/q2-gglasso/data/atacama-demux-subsample/atacama_raw.tsv", sep='\t', index_col = 0)

print("Some columns contain only zeros:", (raw == 0).all().any())

raw = raw.T

In [ ]:
raw

In [ ]:
i = 1
for col in raw.columns:
    # length of ASVs identifier
    if len(col) == 32:
        asv_name = "ASV_{0}".format(i)
        raw.rename(columns={col: asv_name}, inplace=True)

        i += 1

In [ ]:
asv_18 = raw["ASV_18"].values

In [ ]:
asv_51 = raw["ASV_51"].values

In [ ]:
asv_51

In [ ]:
raw = raw.T

In [ ]:
percentage_zeros = (raw == 0).sum(axis=1) / len(raw.columns) * 100
sorted_ixs = percentage_zeros.sort_values(ascending=False).index
a = percentage_zeros.sort_values(ascending=True)

In [ ]:
a.head(40)

In [ ]:
raw_sorted = raw.loc[sorted_ixs]
raw_sorted

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(20,20)})
sns.heatmap(raw_sorted, annot=False, fmt="g", cmap='viridis', mask=raw_sorted==0)

plt.show()

In [ ]:
# calculate percentage of zeros for each row
percentage_zeros = (raw == 0).sum(axis=1) / len(raw.columns) * 100



print(percentage_zeros.to_string())

In [ ]:
asv_18 = raw.iloc[18,:]
asv_18

In [ ]:
asv_51 = raw.iloc[51,:]
asv_51

In [ ]:
# x = np.linspace(0, 10, 30)
# y = np.sin(x)

plt.plot(asv_18, asv_51, 'o', color='black');

In [ ]:
df = raw.copy()

In [ ]:
columns

In [ ]:
raw.T

In [ ]:
pattern = r'(\w+\.\d+\.\d+)' 
columns = [col for col in df.columns if re.match(pattern, col)]

for prefix in set(col[:-4] for col in columns):
    pref1 = f"{prefix}.1.1"
    pref2 = f"{prefix}.1.2"
    pref3 = f"{prefix}.1.3"
    pref_avg = f"{prefix}.agg"

    if pref1 in df.columns and pref2 in df.columns and pref3 in df.columns:
        # create a boolean mask to identify rows where pref1 has value zero and pref2 or pref3 has non-zero
        mask = (df[pref1] == 0) & ((df[pref2] != 0) | (df[pref3] != 0))
        
        # create a new column pref_avg with default value np.nan
        df[pref_avg] = 0

        # assign the maximum value among pref2 and pref3 to pref_avg on rows where the mask is True
        df.loc[mask, pref_avg] = df.loc[mask, [pref2, pref3]].max(axis=1)
        
        # drop pref1, pref2, and pref3
        df.drop([pref1, pref2, pref3], axis=1, inplace=True)

    elif pref2 in df.columns and pref3 in df.columns:
        # create a boolean mask to identify rows where pref2 has value zero and pref3 has non-zero
        mask = (df[pref2] == 0) & (df[pref3] != 0)

        # create a new column pref_avg with default value np.nan
        df[pref_avg] = 0

        # assign the value of pref3 to pref_avg on rows where the mask is True
        df.loc[mask, pref_avg] = df.loc[mask, pref3]
        
        # drop pref2 and pref3
        df.drop([pref2, pref3], axis=1, inplace=True)

# rename YUN1005.1.1 to YUN1005.1
df.rename(columns={"YUN1005.1.1": "YUN1005.agg"}, inplace=True)

In [ ]:
df.T

In [ ]:
meta = pd.read_csv('data/acm_meta.tsv', sep='\t', index_col = 0)

# select only numeric features
meta = meta.loc[:, meta.iloc[0, :] != 'categorical']
meta = meta.apply(pd.to_numeric, errors='coerce')
# drop QIIME2 header
meta = meta.iloc[1:]

meta_agg = meta.loc[columns]

# Group the rows by the first part of the index
grouped = meta_agg.groupby(meta_agg.index.str.split('.').str[0])

# Calculate the mean of each group
means = grouped.mean()

# # Create a new DataFrame with the means and the old indices
meta_agg = pd.DataFrame(means.values, index=means.index, columns=means.columns)
meta_agg.index = meta_agg.index + '.agg'

meta  = pd.concat([meta, meta_agg])

In [ ]:
# calculate percentage of zeros for each row
percentage_zeros_1 = (df == 0).sum(axis=0) / len(df.columns) * 100

# drop rows with more than 80% of zeros
df = df[percentage_zeros <= 95]

percentage_zeros_1

In [ ]:
print("Some columns contain only zeros:", (df == 0).all().any())